In [1]:
%load_ext watermark
%watermark

ModuleNotFoundError: No module named 'watermark'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [5]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [1244]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim      = []

for i,j in enumerate(file_path):
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
#         r2_v2_rlim.append(data['r2_v2_rlim'])

('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_tem

In [8]:
def tanh_to10(x,x0,K,theta):
    return 10**( K*(1+tanh(theta*(x-x0))) )
def tanh_plus_to10(x,x0,K,theta,a,b):
    return 10**( K*(1+tanh(theta*(x-x0))) + a*(x)+b )

def gen_logi_to10(x,A,K,C,B,M,nu):
    return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )

def func2_to10(t, t0, A, B,  C, D):
    return 10**( A*(t-t0)/(C+(t-t0)**2)**D + B )
def func2abs_to10(t, t0, A, B,  C, D):
    return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )
def func2ter_to10(t, t0, A, B, D):
    return 10**( A*(t-t0)**(2*D) / (1+(t-t0)**2)**D + B )
#def func2bis_to10(t, t0, A, B,  C, D):
#    tau = t-t0
#    return 10**( (A-B)*(tau) / (C+(tau**2)**(D/2))**(1/D) + B )
def func3_to10(t, t0, A, B,  C):
    return 10**( A*(t-t0) / (C+(t-t0)**2) + B )
def func4_to10(t, t0, A, B, C):
    return 10**( A*(t-t0) / (C+abs(t-t0)) + B )
def func5_to10(t, t0, A, B, C, k):
    return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B )
def func6_to10(t, t0, A, C, k):
    return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) )
def func7_to10(t, t0, A, B, k):
    return 10**( A*(t-t0) / (1+abs(t-t0)**k)**(1/k) + B)

In [25]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+(t-t0)**2)**D + B
def func2abs(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+abs(t-t0))**D + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B

In [1183]:
def func_lin(t,a,b):
    return a*t+b

In [254]:
cm = pylab.get_cmap('turbo')

In [364]:
figure('coucou').clf()
x  = time[5]
y  = T_aux_avg[5]
semilogy(x,y)

In [1255]:
# Temperature as a function of time
to_fit = 22 #11
x  = time[to_fit]
y  = T_aux_avg[to_fit]

sigma_size = r2_v2_rlim[to_fit][0][:len(y)] + r2_v2_rlim[to_fit][1][:len(y)] + r2_v2_rlim[to_fit][2][:len(y)]

figure('testfitfunctions',clear='True')
#xlim(0.8,6)

#plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(x*1e3, y,label=r'$T filt$',color='C0')
plot(x*1e3, y_filt,label=r'$T filt$',color='C1',ls=':')
grid()
legend()

figure('T fct size',clear='True')
b, a = signal.butter(8, 0.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(y,sigma_size,label=r'$T filt$',color='C0')
plot(y_filt,sigma_size,label=r'$T filt$',color='C1',ls=':')
grid()
legend()

IndexError: list index out of range

In [1241]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
3 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
4 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
5 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
6 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
7 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
8 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
9 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_

In [1385]:
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
Urf = [0, 0, 0, 0, 0, 55.35111, 57.40115, 59.45119, 61.50124, 63.55128, 65.60132, 67.65136, 69.70140, 71.75144,0, 0, 0, 0, 0, 64.6, 62.5]
q = [0, 0, 0, 0, 0, 0.54, 0.56,  0.58,  0.6, 0.62,  0.64,  0.66,  0.68,  0.7, 0, 0, 0, 0, 0, 0.63, 0.61]
Udc = [0, 0, 0, 0, 0, 11.10109, 12.0829, 13.12915, 14.24497, 15.43671, 16.71181, 18.07917, 19.54956, 21.13623, 0, 0, 0, 0, 0, 16.0, 14.8]

omega_z_2 = ( 2*pi*100e3 )**2 * array(Udc)
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2
print(sqrt(omega_r_2[my_try])/2/pi)

fs = 1/diff(x)[1]

357173.32014723605


In [1387]:
import matplotlib.ticker as ticker

In [1400]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 2  #11
temp_to_end = 75

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*1.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log10(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
print('func5',popt)

figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax2 = subplot(111)
ax2.grid()
ax2.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax2.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

y_fit5 = func5(x[start_for_fit:end_fit],*popt)
ax2.semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax2.yaxis.set_major_locator(maj_loc)
ax2.yaxis.set_minor_locator(min_loc)

ax2.set_xlabel('t [ms]')
ax2.set_ylabel('T [K]')
# ax2.set_xlim(-0.3,5.6)
# ax2.set_ylim(2e-3,300)
tight_layout()
# savefig(figname+'.eps',dpi=300)

file to analyze
> 2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
func5 [ 0.0624296   3.66795958 -0.01062933  0.04522017  0.14764834]


In [1393]:
10**0.056677

1.1394020591841805

In [1401]:
# Fit all curves
# Smooth data temperature
# guess first parameter set
# fit smoothed data up to threshold temperature
# much more consistent than below

popt_smooth = []
temp_to_end = [100, 100, 75, 50, 50, 50, 50, 50, 50,
               75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
               100, 100, 100, 100, 100, 100, 100, 100, 75, 150, 100] # Temperature to stop the fit
# 100, 100, 100, 100, 100, 100, 100, 100, 100   75, 75, 75, 75, 75, 75, 75, 75, 75
start_for_fit = 200
T_crit = 0.5
print('file to analyze')
not_fitted = 0
for to_fit,j in enumerate(file_path):
    if to_fit!=0 and to_fit!=9 and to_fit!=10 and to_fit!=11:
    # retrive time and temperature from the curve
        print('>',to_fit,j)
        my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
        x  = time[to_fit]-1e-3
        y  = T_aux_avg[to_fit]
                
        # Smoothing signal with zero delay shift fit
        max_f_r = sqrt(omega_r_2[my_try])/2/pi
        b, a = signal.butter(8,  max_f_r/fs*1.01)
        y_filt = signal.filtfilt(b, a, y, padlen=150)

        # Fit parameters first guesses
        t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
        B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
        A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)
        
        # select index of temp where stop the fit
        # I stop the fit when T reaches a given T
        index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
        end_fit = index_100K

        # set initial guess of fit parameters
        p0 = [t0_first_guess,A0,B0,0.01,0.3]
        try:
            # proper fit of the smoothed curve limiter up to threshold T
            popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                                          log10(y_filt[start_for_fit:end_fit]),
                                   p0, maxfev= 25000,
                                   bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
            print(popt)
            popt_smooth.append(popt)
        except RuntimeError:
            print('Optimal parameters not found: Number of calls to function has reached maxfev')
            not_fitted += 1
    else:
        print('>',to_fit,j)
        print('Data skipped')
        not_fitted += 1
print('total amount of non fitted curves',not_fitted)

file to analyze
> 0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
Data skipped
> 1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
[ 0.18491963  4.00996393 -0.02050412  0.05229461  0.117688  ]
> 2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
[ 0.06242858  3.65060518 -0.01115949  0.04495579  0.14867785]
> 3 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
[ 0.02173361  3.59492208 -0.06978041  0.04214644  0.15910297]
> 4 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
[ 0.00737913  3.59732554 -0.14105181  0.03950778  0.1737245 ]
> 5 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
[ 0.00308311  2.78324138 -0.22485179  0.01919542  0.25681643]
> 6 /home/adrien/Documents/Programmes

In [792]:
figure('coucou2')
plot(array(popt_rework[1])[:,0])
array(popt_rework[1])[:,0]

array([0.01279912, 0.00127116, 0.01150881, 0.00515209, 0.00302405,
       0.00208403, 0.00168962, 0.00128867])

In [1404]:
popt_rework = [popt_smooth[:8],
              popt_smooth[8:16], # 7:15
              popt_smooth[16:]]
#  3     4     5     6     7     8     9     10    11    12    13
# 31.0, 42.0, 55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8
#             0.54  0.56  0.58  0.6   0.62  0.64  0.66  0.68  0.7
Urf_rework = [[57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8],
             [57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8], # 57.4, 59.5, 
             [55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8, 64.6, 62.5]]
q_rework = [[0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7],
             [0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7], # 57.4, 59.5, 
             [0.54, 0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7, 0.63, 0.61]]
N_rework = [256, 512, 1024]

for k,l in enumerate(N_rework):
    print(f'fit_params_{l:04d}ions')
    np.savez( f'fit_params_{l:04d}ions', fit_param=popt_rework[k], Urf=Urf_rework[k], N=N_rework[k])

# func5(t, t0, A, B, C, k)
# with load('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/fit_params_0256ions.npz') as data:
#     a = data['N']
# print(a)

fit_params_0256ions
fit_params_0512ions
fit_params_1024ions


In [1405]:
figure('fit_parameters',clear=True)
ax1 = subplot(151)
for k in range(3):
    ax1.plot(Urf_rework[k],array(popt_rework[k])[:,0]*1e3-1,color=cm(k/5),ls=':',marker='P',label=N_rework[k])
    ax1.set_xlabel('$U_{RF}$ [V]')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
    ax2 = subplot(152)
    ax2.plot(Urf_rework[k],10**array(popt_rework[k])[:,1],color=cm(k/5),ls=':',marker='P',label=N_rework[k])
    ax2.set_xlabel('$U_{RF}$ [V]')
    ax2.set_ylabel('$A$ [K]')
    ax2.grid()
    ax2.legend(title='N',fontsize=12)
    ax3 = subplot(153)
    ax3.plot(Urf_rework[k],10**array(popt_rework[k])[:,2],color=cm(k/5),ls=':',marker='P',label=N_rework[k])
    ax3.set_xlabel('$U_{RF}$ [V]')
    ax3.set_ylabel('$10^B$ [K]')
    ax3.grid()
    ax3.legend(title='N',fontsize=12)
    ax4 = subplot(154)
    ax4.plot(Urf_rework[k],array(popt_rework[k])[:,3]*1e3,color=cm(k/5),ls=':',marker='P',label=N_rework[k])
    ax4.set_xlabel('$U_{RF}$ [V]')
    ax4.set_ylabel('$C$ [ms]')
    ax4.grid()
    ax4.legend(title='N',fontsize=12)
    ax5 = subplot(155)
    ax5.plot(Urf_rework[k],array(popt_rework[k])[:,4],color=cm(k/5),ls=':',marker='P',label=N_rework[k])
    ax5.set_xlabel('$U_{RF}$ [V]')
    ax5.set_ylabel('$k$ [1]')
    ax5.grid()
    ax5.legend(title='N',fontsize=12)
tight_layout()

In [1346]:
cm = pylab.get_cmap('tab10')

In [1366]:
figname = 'fit_parameter_t0'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.semilogy(q_rework[k],array(popt_rework[k])[:,0]*1e3-1,
                 color=cm(k/10),ls='',marker='P',ms=10,mew=1,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, array(q_rework[k]),log10(array(popt_rework[k])[:,0]*1e3-1),maxfev= 15000)
    ax1.semilogy(array(q_rework[k]),10**func_lin(array(q_rework[k]),popt[0],popt[1]),
                 ls='--',color=cm(k/10))
    ax1.set_xlabel('$q_{x}$')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=18)
ax1.grid(visible=True, which='major', color='0.2', linestyle='-')
ax1.grid(visible=True, which='minor', color='0.7', linestyle='--')
tight_layout()
savefig(figname+'.png',dpi=300)
    
fig = figure('fit_parameter_t0_bis',clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3-1)**(-1/2),color=cm(k/4),
             ls=':',marker='P',ms=10,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3-1)**(-1/2),maxfev= 15000)
    ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),color=cm(k/3))
    ax1.set_xlabel('$q_x$')
    ax1.set_ylabel('$t_0^{-1/2}$ [ms$^{-1/2}$]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
tight_layout()

In [1408]:
fig = figure('fit_parameter_B',clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(Urf_rework[k],array(popt_rework[k])[:,2]*1e3,color=cm(k/5),ls=':',marker='P',ms=10,mew=1,label=N_rework[k])
    ax1.set_xlabel('$U_{RF}$ [V]')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
tight_layout()
    
figname = 'fit_parameter_B_bis'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(array(q_rework[k]),10**array(popt_rework[k])[:,2],
             color=cm(k/10),ls='',marker='P',ms=10,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, array(q_rework[k]),(10**array(popt_rework[k])[:,2]),maxfev= 15000)
    ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),
             color=cm(k/10),ls='--')
    ax1.set_xlabel('$q_x$')
    ax1.set_ylabel('$B=T(t=t_0)$ [K]')
    ax1.grid()
    ax1.legend(title='N',fontsize=18)
tight_layout()
savefig(figname+'.png',dpi=300)